# Python Data, 2025

---

* [Manipulace se sloupci](#Spojov%C3%A1n%C3%AD-dat),
* [Spojování dat](#Spojov%C3%A1n%C3%AD-dat),
    - [spojování s CONCAT](#Spojování-s-CONCAT),
    - [spojování s CONCAT a JOIN](#concat-a-volitelný-argument-join),
    - [spojování s metodou APPEND](#Spojování-pomocí-metody-APPEND),
    - [spojování s MERGE](#Spojování-pomocí-merge),
    - [spojování s metodou JOIN](#Spojení-pomocí-indexů,-join),
* [Agregace](#Agregace),
    - [jednoduchá agregace](#Jednoduchá-agregace),
    - [seskupování GROUPBY](#Seskupování-groupby),
    - [agregace](#Agregace-s-metodami),
    - [filtrování](#Filtrování),
    - [transformace](#Transformace),
    - [apply](#Metoda-apply),
* [Pivot tabulka](#Pivot-tabulky),
    - [úvodní motivace](),
    - [syntaxe tabulky](),
    - [doplňující možnosti]().
---

<br>

## Manipulace se sloupci

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse4.mm.bing.net%2Fth%3Fid%3DOIP.tzaGc-M5pl0DP6pGu-H6WgHaHa%26pid%3DApi&f=1&ipt=39909793c49bf980b8dbceaabe312870e1446f58fba1e6b430630cfbddd2b9c4&ipo=images" width="200" style="margin-left:auto; margin-right:auto"/>



Jakmile se dovedeš zorientovat v datových strukturách knihovny `pandas`, můžeš začít upravovat jejich obsah:
1. **Přejmenovat** sloupce,
2. **vytvořit** nové sloupce,
3. **odstranit** existující sloupce.

<img src="https://imgs.search.brave.com/gKVKM0gMPdcdyI_mt5KyWfGrx-nGAFVdDOrSLlHo5wo/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9pbWFn/ZXM3Lm1lbWVkcm9p/ZC5jb20vaW1hZ2Vz/L1VQTE9BREVEODAw/LzY0ZDE1NGQ4ZGQy/MTMuanBlZw" width="300" style="margin-left:auto; margin-right:auto"/>

### Přejmenování sloupců

---

Pokud budeš někdy zápasit se špatně pojmenovaným sloupcem nebo sloupci, přejmenuj je pomocí metody `rename`.

In [37]:
from pandas import DataFrame

In [38]:
uzivatele = {'jmeno': ['Matouš', 'Marek', 'Lukáš', 'Jan'],
             'age': [23, 25, 27, 29],
             'vaha': [65, 70, 75, 80],
             'vyska': [160, 170, 180, 190]}

In [39]:
uzivatele_df = DataFrame(uzivatele)

In [40]:
uzivatele_df

,jmeno,age,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


V ukázce výš tě omezuje **jazyková nekonzistence** mezi sloupečky.

Pomocí metody `rename` a argumentu `columns`, můžeš pomocí `dict` anotace vybrat, co a čím přejmenovat:

In [41]:
uzivatele_df.rename(columns={"age": "vek"})

,jmeno,vek,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


Při další kontrole zjistíš, že **nové jméno sloupečku zmizelo**:

In [42]:
uzivatele_df

,jmeno,age,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


<br>

Celou situaci můžeš řešit dvěma způsoby:
1. Volitelný argument `inplace`,
2. kopie původního `DataFrame` objektu.

In [43]:
prejmenovany_sloupec_df = uzivatele_df.rename(columns={"age": "vek"})

In [44]:
prejmenovany_sloupec_df

,jmeno,vek,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


In [45]:
uzivatele_df

,jmeno,age,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


In [48]:
uzivatele_df.rename(columns={"age": "vek"}, inplace=True)  # ⚠️ Za cenu rizika!⚠️

In [49]:
uzivatele_df

,jmeno,vek,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


Opatrně na práci s argumentem `inplace`.

Jeho použití vždy důkladně zvaž, ať neztratíš zdrojová data.

<br>

Přejmenovat můžeš i **více sloupců** současně:

In [50]:
nova_jmena_sloupcu = {  # CZ > EN
    "jmeno": "first_name",
    "vek": "age",
    "vaha": "weight",
    "vyska": "height"
}

In [51]:
uzivatele_df.rename(columns=nova_jmena_sloupcu, inplace=True)

In [52]:
uzivatele_df

,first_name,age,weight,height
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190




<img src="https://imgs.search.brave.com/Q3iNWCeKLnizjFbTrtaUEiEcXWvAqdNi_cWO1vwkJo0/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9pbWFn/ZXMuaWNvbi1pY29u/cy5jb20vMTE1NC9Q/TkcvOTYvMTQ4NjU2/NDQwNy1wbHVzLWdy/ZWVuXzgxNTIxLnBu/Zw" width="100" style="margin-left:auto; margin-right:auto"/>

### Přidávání sloupců do DataFrame

---

Potřebuješ doplnit další sloupec. Nebo upravit data v jednom sloupci a výsledek zapsat do dalšího.

Nejčastěji můžeš pracovat **na přidávání nových sloupců** těmito třemi způsoby:
1. **Slovníková anotace** pomocí klíče a hodnoty,
2. výsledek pomocí operace **mezi dvěma sloupci**,
3. pomocí metody `apply`.

#### Jako klíč a hodnota

---

Takový způsob použiješ, pokud máš nachystané hodnoty pro nový sloupec:

In [53]:
nova_jmena_sloupcu = {  # EN > CZ
    "first_name": "jmeno",
    "age": "vek",
    "weight": "vaha",
    "height": "vyska"
}

In [54]:
uzivatele_df.rename(columns=nova_jmena_sloupcu, inplace=True)

In [55]:
uzivatele_df

,jmeno,vek,vaha,vyska
0,Matouš,23,65,160
1,Marek,25,70,170
2,Lukáš,27,75,180
3,Jan,29,80,190


In [56]:
uzivatele_df['prijmeni'] = ("Holinka", "Párek", "Holinka", "Novák")

In [18]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni
0,Matouš,23,65,160,Holinka
1,Marek,25,70,170,Párek
2,Lukáš,27,75,180,Holinka
3,Jan,29,80,190,Novák


In [57]:
uzivatele_df.iloc[1]

jmeno       Marek
vek            25
vaha           70
vyska         170
prijmeni    Párek
Name: 1, dtype: object

<br>

#### Kombinace existujících sloupců

---

Tento způsob lze použít, pokud chceš vytvořit nový sloupec pomocí operací mezi existujícími sloupci:

In [20]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni
0,Matouš,23,65,160,Holinka
1,Marek,25,70,170,Párek
2,Lukáš,27,75,180,Holinka
3,Jan,29,80,190,Novák


In [60]:
uzivatele_df['BMI'] = round(uzivatele_df['vaha'] / ((uzivatele_df['vyska'] / 100) ** 2), 1)

In [61]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI
0,Matouš,23,65,160,Holinka,25.4
1,Marek,25,70,170,Párek,24.2
2,Lukáš,27,75,180,Holinka,23.1
3,Jan,29,80,190,Novák,22.2


#### Metoda `apply`

---

Pokud potřebuješ aplikovat uživ. funkci **pro stávající sloupec**, využiješ metodu `apply`:

In [23]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI
0,Matouš,23,65,160,Holinka,25.4
1,Marek,25,70,170,Párek,24.2
2,Lukáš,27,75,180,Holinka,23.1
3,Jan,29,80,190,Novák,22.2


In [62]:
def vytvor_email(prijmeni, domena= "gmail.com"):
    return f"{prijmeni.lower()}@{domena}"

In [63]:
vytvor_email(prijmeni='holinka')

'holinka@gmail.com'

In [64]:
uzivatele_df['email'] = uzivatele_df['prijmeni'].apply(vytvor_email)

In [65]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,email
0,Matouš,23,65,160,Holinka,25.4,holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,novák@gmail.com


<br>

Pokud budeš potřebovat pracovat **s více sloupečky**:

In [66]:
from pandas import Series

In [67]:
def vytvor_novy_email(radek, domena= "gmail.com"):
    return f"{radek['jmeno'].lower()}.{radek['prijmeni'].lower()}@{domena}"

In [68]:
jeden_uzivatel = DataFrame({'jmeno': ['Matous'], 'prijmeni': ['Test']})

In [69]:
jeden_uzivatel.iloc[0]

jmeno       Matous
prijmeni      Test
Name: 0, dtype: object

In [70]:
vytvor_novy_email(radek=jeden_uzivatel.iloc[0])

'matous.test@gmail.com'

In [71]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,email
0,Matouš,23,65,160,Holinka,25.4,holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,novák@gmail.com


In [72]:
uzivatele_df['dalsi_email'] = uzivatele_df.apply(vytvor_novy_email, axis=1)

In [73]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,email,dalsi_email
0,Matouš,23,65,160,Holinka,25.4,holinka@gmail.com,matouš.holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,párek@gmail.com,marek.párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,holinka@gmail.com,lukáš.holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,novák@gmail.com,jan.novák@gmail.com


<br>

Nepovinný argument `axis` v ohlášení říká, že chce aplikovat metodu **na řádkové hodnoty**.

In [74]:
# uzivatele_df.apply?

In [75]:
# uzivatele_df.head()

Tento způsob lze použít, pokud chceš vytvořit **nový sloupec** pomocí nějaké funkce na jednotlivé řádky nebo sloupce `DataFrame`.

<br>

### Odstraňování sloupečků

---

K odstranění sloupců z `DataFrame` objektu můžeš použít několik metod:
1. pythonovskou funkci `del`,
2. metodu `drop`,
3. metodu `pop`.

In [76]:
from pandas import DataFrame

In [77]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,email,dalsi_email
0,Matouš,23,65,160,Holinka,25.4,holinka@gmail.com,matouš.holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,párek@gmail.com,marek.párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,holinka@gmail.com,lukáš.holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,novák@gmail.com,jan.novák@gmail.com


In [79]:
'dalsi_email' in uzivatele_df.columns

True

In [80]:
uzivatele_df.index

RangeIndex(start=0, stop=4, step=1)

In [ ]:
# uzivatele_df.drop?

In [81]:
uzivatele_df = uzivatele_df.drop('dalsi_email', axis=1)

In [82]:
uzivatele_df

,jmeno,vek,vaha,vyska,prijmeni,BMI,email
0,Matouš,23,65,160,Holinka,25.4,holinka@gmail.com
1,Marek,25,70,170,Párek,24.2,párek@gmail.com
2,Lukáš,27,75,180,Holinka,23.1,holinka@gmail.com
3,Jan,29,80,190,Novák,22.2,novák@gmail.com


<br>

**🧠 CVIČENÍ 🧠, zpracuj zadané body v úloze.**

Máš DataFrame `zamestnanci` s informacemi o zaměstnancích tvé firmy.

Potřebuješ následující:
1. **přidat sloupec** s názvem `plat_na_hodinu`, který bude vypočítán jako průměr platu zaměstnanců dělený počtem odpracovaných hodin.
2. Sloupec `plat_na_hodinu` zaokrouhli **na celé číslo**,
3. **přejmenuj sloupec** `jmeno` na `cele_jmeno`,
4. **odstraň sloupec** `id`.
5. ☢️ **vytvoř funkci** `preved_datum`, která změní formát data ve sloupci `datum_narozeni` z `YYYY-MM-DD` na `DD/MM/YYYY`,

In [83]:
from datetime import datetime

import pandas as pd

In [84]:
zamestnanci_df = pd.DataFrame({'id': [1, 2, 3, 4],
                            'jmeno': ['Jan Novák', 'Petr Soukup', 'Marie Horáková', 'Jana Svobodová'],
                            'datum_narozeni': ['1990-05-15', '1985-12-10', '1992-07-25', '1988-03-20'],
                            'plat': [40_000, 45_000, 35_000, 50_000],
                            'odprac_hodiny': [160, 180, 150, 200]})

<details>
    <summary>▶️ Řešení</summary>
    
```python
zamestnanci['plat_na_hodinu'] = round(zamestnanci['plat'] / zamestnanci['odprac_hodiny'])

zamestnanci = zamestnanci.rename(columns={'jmeno': 'cele_jmeno'})

zamestnanci = zamestnanci.drop(columns=['id'])


def preved_datum(datum_narozeni, vstupni_format='%Y-%m-%d', vystupni_format='%d/%m/%Y'):
    dt = datetime.strptime(datum_narozeni, vstupni_format)
    prevedene_datum = dt.strftime(vystupni_format)
    return prevedene_datum


# zamestnanci['datum_narozeni'] = pd.to_datetime(zamestnanci['datum_narozeni']).dt.strftime('%d/%m/%Y')  #  I. řešení
# zamestnanci['datum_narozeni'] = zamestnanci['datum_narozeni'].apply(preved_datum)                      # II. řešení
```
</details>

---



<img src="https://imgs.search.brave.com/avW2sADDSquCer8A7CT2t7rtq5ea-z8nvmRLilPc4-U/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9kZ2Vp/dTNmejI4Mng1LmNs/b3VkZnJvbnQubmV0/L2cvbC9sZ2ZwMTgz/OStqb2luLXVzLW9y/LWRpZS1kYXJ0aC12/YWRlci1zdGFyLXdh/cnMtMzB0aC1hbm5p/bnZlcnNhcnktcG9z/dGVyLmpwZw" width="160" style="margin-left:auto; margin-right:auto"/>

<br>

## Spojování dat

---

Takovými operacemi si můžeš představit jednoduché operace jako **spojování** (*konkatenace*) **dvou a více datasetů**, tedy tabulek.

Obdoba pro složitější *JOINY* jako u databází a jejich tabulek.

`pandas` obsahuje různé *funkce* a *metody*, které umožňují tento způsob práce:

* ✅ funkce `concat`,
* ✅ funkce `merge`,
* ⛔️ metoda `append` (*outdated*),
* ✅ metoda `join`.

### Spojování s CONCAT

---

Funkce `concat()` v `pandas` umožňuje spojit **dva nebo více**:
1. **sloupečků** tabulky,
    - *horizontálně*, po sloupcích,
    - *vertikálně*, po Indexech,
3. **tabulek** samotných,
    - *horizontálně*, po sloupcích,
    - *vertikálně*, po Indexech,

<br>

Použití funkce `concat()` spočívá **v předání seznamu objektů**, které chceš spojit, a parametru `axis`, který určuje osu, podle které se májí objekty spojit:

#### Spojení sloupečků, CONCAT

---

In [ ]:
from pandas import concat, Series, DataFrame

In [ ]:
sloupecek_1 = Series(['A', 'B', 'C'], index=[1, 2, 3])
sloupecek_2 = Series(['D', 'E', 'F'], index=[4, 5, 6])

In [ ]:
sloupec_1

In [ ]:
sloupec_2

<br>

##### Spojení **vertikální** (Indexy navazují, tabulky "pod sebou"):

---

In [ ]:
# spojene_sloupce_vertikalne = concat(sloupec_1, sloupec_2, axis=0)

<br>

Nesprávné spojení, potřeba nahlédnout do popisku funkce:

In [ ]:
# concat?

In [ ]:
spojene_sloupce_vertikalne = concat((sloupec_1, sloupec_2), axis=0)

Nyní spojujeme sekvenci `tuple`, což už spadá mezi iterovatelné datové typy.
<br>

Jaký parametr je `axis` a jakých může nabývat hodnot:

In [ ]:
# concat?

In [ ]:
spojene_sloupce_vertikalne

<br>

##### Spojení **horizontální** (vedle sebe, přidání sloupečku):

---

In [ ]:
sloupce_spojene_horizontalne = concat([sloupec_1, sloupec_2], axis=1)

In [ ]:
sloupce_spojene_horizontalne

<br>

#### Spojení tabulek, CONCAT

---

In [ ]:
uzivatele_tab_1 = {'jmeno': ['Matouš', 'Marek', 'Lukáš'],
                   'vek': [25, 30, 35]}

In [ ]:
uzivatele_tab_2 = {'jmeno': ['Petr', 'Jan', 'Michal'],
                   'vek': [40, 45, 50]}

In [ ]:
uzivatele_tab_1_df = DataFrame(uzivatele_tab_1)
uzivatele_tab_2_df = DataFrame(uzivatele_tab_2)

In [ ]:
uzivatele_tab_1_df

In [ ]:
uzivatele_tab_2_df


#### Jak zapsat spojení 2 různých tabulek?

---

Stejně jako u sloupečků, nezapomínat na poziční argumenty (závorka).

Stejně tak, vybrat správnou osu.

In [ ]:
spojene_tab_df = concat([uzivatele_tab_1_df, uzivatele_tab_2_df], axis=0)

In [ ]:
spojene_tab_df

#### Co s duplicitními Indexy?

---

Tady se ovšem **zduplikovali hodnoty** některých indexů.

Ty je potřeba opravit tímto postupem:
1. **Vytvořím nový sloupeček** pro Indexy,
2. **odstraním starý sloupeček** s duplicitami.

In [ ]:
spojene_tab_df = spojene_tab_df.reset_index()  # Přidá nový Index

<br>

Metoda `reset_index` přidá nový, defaultní sloupec s pomocným Indexem.

In [ ]:
spojene_tab_df

In [ ]:
spojene_tab_df = spojene_tab_df.drop("index", axis=1)

In [ ]:
spojene_tab_df

<br>

Kontrola nového rozsahu **pro Index**:

In [ ]:
spojene_tab_df.index

<br>

Pozůstatky sloupečků:

In [ ]:
spojene_tab_df.columns

<br>

Nastavení jména nového Indexu:

In [ ]:
spojene_tab_df.index.names

<br>

Doplníš jméno **pro Index**:

In [ ]:
spojene_tab_df.index.names = ["Nový Index"]

In [ ]:
spojene_tab_df.index.names

In [ ]:
spojene_tab_df.index

In [ ]:
spojene_tab_df

Pro odchytávání **duplicitních indexů** můžeš doplnit parametr `verify_integrity=True`, případně pokud je irelevantní, ignorovat jej úplně `ignore_index=True`.

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí CONCAT**

In [ ]:
from typing import Dict

from pandas import DataFrame

In [ ]:
data_populace = {"City": ["New York", "London", "Tokyo"],
                 "Population": [8.4, 9.0, 13.9]}

data_plocha = {"City": ["New York", "London", "Tokyo"],
               "Area": [783, 1572, 2194]}

data_kontinent = {"City": ["New York", "London", "Tokyo"],
                  "Continent": ["North America", "Europe", "Asia"]}

# ❓ Vytvoř funkci "spoj_data_mest", která spojí 3 předchozí tabulky❓

<details>
    <summary>▶️ Řešení</summary>
    
```python
from pandas import DataFrame, concat

data_populace = {
    "City": ["New York", "London", "Tokyo"],
    "Population": [8.4, 9.0, 13.9],
}

data_plocha = {
    "City": ["New York", "London", "Tokyo"],
    "Area": [783, 1572, 2194],
}

data_kontinent = {
    "City": ["New York", "London", "Tokyo"],
    "Continent": ["North America", "Europe", "Asia"],
}

populace_df = DataFrame(data_populace)
plocha_df = DataFrame(data_plocha)
kontinent_df = DataFrame(data_kontinent)


populace_df = DataFrame(data_populace)
plocha_df = DataFrame(data_plocha)
kontinent_df = DataFrame(data_kontinent)


def spoj_data_mest(*args) -> DataFrame:
    data = [arg.set_index('City') for arg in args]
    return concat(data, axis=1)


result = spoj_data_mest(populace_df, plocha_df, kontinent_df)
print(result)
```
</details>

<img src="https://imgs.search.brave.com/wR9sz6BPwR_LCDm6Esd_B9VgUBSbW4wbTPi2T9DrdBc/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly90NC5m/dGNkbi5uZXQvanBn/LzA4LzQwLzM1LzIx/LzM2MF9GXzg0MDM1/MjE1Ml84N1N2T0kx/REM2RElCaHkxc0xz/VnpnYmNIUk1UZno0/cC5qcGc" width="230" style="margin-left:auto; margin-right:auto"/>

### CONCAT a volitelný parametr JOIN

---

V jednoduchých ukázkách, jako jsou ty výše, stačilo **tabulky a sloupečky spojit**.

To prakticky není vždy ideální řešení, protože některé sloupečky se mohou lišit.

Třeba, pokud mají různé jména.

In [ ]:
from pandas import DataFrame, concat

In [ ]:
data_tab_1_df = DataFrame({"A": ["A1", "A2"],
                           "B": ["B1", "B2"],
                           "C": ["C1", "C2"]},
                          index=[1, 2])

In [ ]:
data_tab_2_df = DataFrame({"B": ["B3", "B4"],
                           "C": ["C3", "C4"],
                           "D": ["D3", "D4"]},
                          index=[3, 4])

In [ ]:
data_tab_1_df

In [ ]:
data_tab_2_df


#### Horizontální spojení dvou tabulek

---

In [ ]:
vystup_tab_df = concat([data_tab_1_df, data_tab_2_df])  # axis=0

In [ ]:
vystup_tab_df

Pokud některá data chybějí, jsou automaticky vyplněná **neznámými hodnotami**.

<br>

Řešením takové situace můžeš být zavedení jiného **způsobu spojování**.

Tedy nepoužívat defaultní argument `join='outer'`, ale `join='inner'`:

In [ ]:
vystup_bez_nan_df = concat([data_tab_1_df, data_tab_2_df],
                           join='inner',
                           axis=0)

In [ ]:
vystup_bez_nan_df

Použití `outer` odpovídá implicitnímu chování funkce `concat`:

In [ ]:
vystup_vc_nan_df = concat([data_tab_1_df, data_tab_2_df],
                           join='outer',
                           axis=0)

In [ ]:
vystup_vc_nan_df

#### 📝 Souhrn k funkci `concat`

---

| Situace                                             | Použiješ                   | join?               |
| --------------------------------------------------- | -------------------------- | ------------------- |
| Spojuješ tabulky se stejnými sloupci pod sebe       | `pd.concat([...])`         | **nepotřebuješ**    |
| Spojuješ tabulky vedle sebe podle indexu            | `pd.concat([...], axis=1)` | volitelně           |
| Máš různé indexy a chceš spojit jen průnik          | `pd.concat([...], axis=1)` | `join='inner'`      |
| Chceš **zachovat vše**, i když něco chybí           | `pd.concat([...], axis=1)` | `join='outer'`      |
| Chceš sloučit podle **konkrétního sloupce** | ❌ `concat` nestačí         | 👉 použij `merge()` |


<img src="https://imgs.search.brave.com/IZ8jkD1GaDV2tXwgwzM1Xawk-jBMejygby1KcermQp8/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly90NC5m/dGNkbi5uZXQvanBn/LzA4LzQ2LzQ3LzE1/LzM2MF9GXzg0NjQ3/MTU2MF95Y281RnhM/UG5RR1c4NW1OWHBD/UmcxUWptSzRiMnp3/eS5qcGc" width="200" style="margin-left:auto; margin-right:auto"/>



### Spojování pomocí `MERGE`

---

Další funkcí pro spojování `DataFrame` objektů je `merge`.

Tato funkce je vhodná pro spojování DataFrame objektů, které **mají společné sloupce**.

Můžeš lépe zadávat typ spojení (parametr `how='inner' | 'outer' | 'left' | 'join'`).

Dále ti umožní **definovat sloupec, nebo sloupce**, na kterých chceš spojení provést (parametr `on`).

In [ ]:
from pandas import merge

In [ ]:
uzivatele_tab_1 = {'jmeno': ['Alice', 'Bob', 'Charlie', 'David'],
                   'vek': [25, 30, 35, 40],
                   'mesto': ['Brno', 'Praha', 'Plzen', 'Ostrava']}

In [ ]:
uzivatele_tab_2 = {'jmeno': ['Alice', 'David', 'Emma', 'Frank'],
                   'pocet_prijemcu': [100, 200, 150, 250]}

In [ ]:
uzivatele_tab_1_df = DataFrame(uzivatele_tab_1)
uzivatele_tab_2_df = DataFrame(uzivatele_tab_2)

In [ ]:
uzivatele_tab_1_df

In [ ]:
uzivatele_tab_2_df

#### Spojení FULL JOIN

---

Jinými slovy spoj všechny záznamy, **pro konkrétní sloupeček**:

In [ ]:
mergnute_df = merge(uzivatele_tab_1_df,
                    uzivatele_tab_2_df,
                    on='jmeno',
                    how='outer')

In [ ]:
mergnute_df

#### Spojené INNER MERGE

---

In [ ]:
mergnute_inner_df = merge(uzivatele_tab_1_df,
                          uzivatele_tab_2_df,
                          on='jmeno',
                          how='inner')

In [ ]:
mergnute_inner_df

<br>

Zde byla ale jména sloupců na první pohled odlišná.

Ne ve všech situacích to platí:

In [ ]:
df8 = DataFrame({'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
                 'poradi': [1, 2, 3, 4]})

In [ ]:
df9 = DataFrame({'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
                 'poradi': [3, 1, 4, 2]})

Funkce sama **doplní přípony**, aby rozlišila **mezi oběma původními sloupci**.

**Pokud potřebuješ vlastní přípony**, můžeš vyzkoušet volitelný argument pro `suffixes`:

In [ ]:
df8

In [ ]:
df9

In [ ]:
vystup_konflikt = merge(df8, df9, on="jmeno")

In [ ]:
vystup_konflikt

In [ ]:
vystup_vlastni_pripony = merge(df8, df9, on='jmeno', suffixes=('_DF8', '_DF9'))

In [ ]:
vystup_vlastni_pripony

Obecně platí, že pokud potřebuješ *spojit* dva nebo více `DataFrame` objektů podle **společného sloupce nebo sloupců**, použij funkci `merge()`.

Pokud chceš jenom **přidat další řádky nebo sloupce** do existujícího `DataFrame` objektu, použij funkci `concat`.

#### Ukázka MERGE funkce s dopomocí `LEFT`

In [ ]:
uzivatele_tab_1_df

In [ ]:
uzivatele_tab_2_df

In [ ]:
vystup_left_join_df = merge(uzivatele_tab_1_df,
                            uzivatele_tab_2_df,
                            on='jmeno',
                            how='left')

In [ ]:
vystup_left_join_df

V ukázce výš je použitý *left join*.

Tedy ve výsledku uvidíš **celou první** (levou tabulku) a z druhé pouze ty záznamy, které mají ve spojovacím sloupci `jmeno` společnou hodnotu.

In [ ]:
vystup_left_join_obr_df = merge(uzivatele_tab_2_df,
                            uzivatele_tab_1_df,
                            on='jmeno',
                            how='left')

In [ ]:
uzivatele_tab_2_df

In [ ]:
vystup_left_join_obr_df

<img src="https://imgs.search.brave.com/-VgG6K4KBK8o5Oy7foYt_v7sPQTBbdERXhQ_ZqsqYLE/rs:fit:500:0:0:0/g:ce/aHR0cHM6Ly9hLnBp/bmF0YWZhcm0uY29t/LzUwMHgzODQvOWZj/MDU0ZTM5Zi9qb2lu/LXVzLmpwZw" width="300" style="margin-left:auto; margin-right:auto"/>

### Spojení pomocí indexů, `JOIN`

---

Tato metoda slouží k propojení dvou `DataFrame` objektů na základě **jejich indexů nebo hodnot**.

Je velice podobná funkci `merge` ale je přímo součástí `DataFrame` objektu a je snazší ji aplikovat:

In [ ]:
uzivatele_1 = {'jmeno': ['Alice', 'Bob', 'Petr'],
               'vek': [25, 30, 35]}

In [ ]:
uzivatele_2 = {'jmeno': ['Alice', 'Bob', 'Petr'],
               'pocet_prijemcu': [100, 150, 200]}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
df_uzivatele_2.index.names == df_uzivatele_1.index.names == [None]

In [ ]:
uzivatele_1_df = df_uzivatele_1.set_index('jmeno')

In [ ]:
uzivatele_2_df = df_uzivatele_2.set_index('jmeno')

In [ ]:
uzivatele_1_df

In [ ]:
uzivatele_2_df

In [ ]:
vysledek_join_metody = uzivatele_1_df.join(uzivatele_2_df)

In [ ]:
vysledek_join_metody

Hlavní rozdíl mezi těmito funkcemi je způsob určení **sloupce nebo sloupců**, podle kterých se má propojení provést.

1. Metoda `join()` propojuje DataFrame objekty **na základě jejich Indexů**.
2. Funkce `merge()` umožňuje propojit `DataFrame` objekty **na základě hodnoty v jednom nebo více sloupcích**.

## 📑Souhrn ke spojování

---

| Objekt | Kdy jej použít? | Směr spojení |
| :- | :- | :- |
| `concat` | Když chceš poskládat tabulky na sebe (řádky) nebo vedle sebe (sloupce). | Vertikálně / Horizontálně | 
| `merge` |	Když chcete spojit tabulky podle společného klíče/sloupce, podobně jako SQL JOIN. | Horizontálně |
| `join` | Když chcete spojit tabulky podle společného Indexu (ne klíče/sloupce). | Horizontálně |
| `append` | (Zastaralé) Používalo se k přidání řádků, nyní používejte concat. | Vertikálně |

<br>

**🧠 CVIČENÍ 🧠, procvič si spojování**

Tvým úkolem je propojit tyto tabulky podle zadání:
1. Použij spojovací funkci pro spojení objektů typu `DataFrame` , které vytvoříš z proměnných `data_zamestnanci_1` a `data_zamestnanci_2`,
2. použij spojovací funkci, které zadáš vzniklý objekt typu `DataFrame`  z bodu 1. a druhý `DataFrame`, který vytvoříš z proměnné `data_oddeleni` (spoj přes sloupeček `"oddělení_id"`),
3. vytvoř výsledný objekt typu `DataFrame` , který obsahuje pouze zaměstnance z oddělení `IT`.

In [ ]:
data_zamestnanci_1 = {
    'id': [1, 2, 3],
    'jméno': ['Jan', 'Marie', 'Petr'],
    'příjmení': ['Novák', 'Svobodová', 'Dvořák'],
    'oddělení_id': [100, 200, 100]
}

data_zamestnanci_2 = {
    'id': [4, 5],
    'jméno': ['Anna', 'Josef'],
    'příjmení': ['Kopecká', 'Vondráček'],
    'oddělení_id': [200, 100]
}

data_oddeleni = {
    'oddělení_id': [100, 200],
    'název_oddělení': ['IT', 'HR']
}

<details>
    <summary>▶️ Řešení</summary>
    
```python
df_zamestnanci_1 = pd.DataFrame(data_zamestnanci_1)
df_zamestnanci_2 = pd.DataFrame(data_zamestnanci_2)
df_oddeleni = pd.DataFrame(data_oddeleni)

df_vsichni_zamestnanci = pd.concat([df_zamestnanci_1, df_zamestnanci_2], ignore_index=True)

# df1 = pd.concat(
#     [pd.DataFrame(data) for data in [data_zamestnanci_1, data_zamestnanci_2]]
# ).reset_index(drop = True).set_index('id')

df_vsechna_oddeleni = pd.merge(df_vsichni_zamestnanci, df_oddeleni, on="oddělení_id")

df_jen_it = df_vsechna_oddeleni[df_vsechna_oddeleni["název_oddělení"] == "IT"]
```
</details>

## Agregace

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.HtBtNx6dodlNptXV1QGHWQHaG0%26pid%3DApi&f=1&ipt=3a4d1dfa14a495127b4ece5cf689f00ed0c9e0836853358412e18161ae7c505f&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>


*Seskupování* neboli **agregace** jsou procesy, které patří k základní efektivní analýze dat.

### Jednoduchá agregace

---

Přesto, že veškeré základní **statistické údaje** prakticky nabízí metoda `describe()`, můžeš ocenit, když stejnou statistiku můžeš aplikovat **na tebou vybrané objekty**.

Mezi nejjednodušší postupy, jak data analyzovat patří metody jako:
* `sum()`,
* `mean()`,
* `median()`,
* `min()`,
* `max()`.

Všechny tyto metody umožní získat jedno samotné číslo, které ti umožní prohlédnout podstatu zadaného datasetu.

In [ ]:
from pandas import read_csv

In [ ]:
nemovitosti_df = read_csv("../onsite/housing.csv")

In [ ]:
nemovitosti_df.head()

In [ ]:
nemovitosti_df.describe()

In [ ]:
nemovitosti_df.loc[:, "price"].max()

<br>

Často ale není dostačující, prozkoumat data pouze jednoduchých agregačních funkcí.

Další operace, které je potřeba pochopit jsou seskupování dat podle zadaných parametrů.

### Seskupování GROUPBY

---

Metoda `groupby`, původně operace z SQL jazyka, je v rámci knihovny `pandas` všestraný pomocník pro seskupování dat na základě různých kritérií.

In [ ]:
from pandas import DataFrame

In [ ]:
pokus_s_cisly_df = DataFrame(
    {'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
     'číselná hodnota': range(6)},
)

In [ ]:
pokus_s_cisly_df

#### Klasické seskupení podle sloupečku

---

In [ ]:
# pokus_s_cisly_df.groupby?

In [ ]:
pokus_s_cisly_df.groupby("klíč")

Jde opět o tzv. *lazy evaluation* proces, samotný nic neprovede, pouze čeká na pokyn uživatele, který samotnou agregaci spustí.

In [ ]:
type(pokus_s_cisly_df.groupby("klíč"))

Metoda standardně vrací `DataFrameGroupBy`.

<br>

In [ ]:
pokus_s_cisly_df.groupby("klíč").sum()

Metoda `sum()` je pouze jednou z možností, se kterou můžeš pracovat.

<br>

#### Sloupečkové označování

---

Stejně jako `DataFrame` můžeš označovat také *GroupBy* objekty.

In [ ]:
pokus_s_cisly_df

In [ ]:
pokus_s_cisly_df.groupby("klíč")

In [ ]:
pokus_s_cisly_df.groupby("klíč")["číselná hodnota"]

In [ ]:
pokus_s_cisly_df.groupby("klíč")["číselná hodnota"].sum()

<br>

V uplynulé ukázce je zadaná seskupování podle sloupečku `klíč`.

Dále je vybraný pouze konkrétní sloupeček, na který chceš spustit metodu `sum`.

<br>

Pokud potřebuješ nad vybraným objektem provádět některé procesy ručně, můžeš přes *GroupBy* objekt **iterovat**:

In [ ]:
for (klic, hodnota) in pokus_s_cisly_df.groupby("klíč"):
    print(f"Klic: {klic}; Hodnota={hodnota}")

<br>

### Agregace s metodami

---

Kromě jednoduchých agregací, nabízí *GroupBy* řadu další funkcionality.

Jde o metody:
* `aggregate`,
* `filter`,
* `transform`,
* `apply`.

#### Metoda AGGREGATE

---

In [ ]:
import numpy

In [ ]:
rng = numpy.random.RandomState(0)

In [ ]:
from pandas import DataFrame

In [ ]:
pokus_s_cisly_df = DataFrame(
    {'klíč': ['A', 'B', 'C', 'A', 'B', 'C', 'C'],
     'data_1': range(7),
     'data_2': rng.randint(0, 10, 7)}
)

In [ ]:
pokus_s_cisly_df

Statistické metody určitě nabízí spousty pomůcek.

Objekt typu *GroupBy* umí pracovat také s funkcemi, stringy a celými listy.

In [ ]:
pokus_s_cisly_df.groupby("klíč").aggregate("sum")

In [ ]:
pokus_s_cisly_df.groupby("klíč").aggregate(numpy.median)

In [ ]:
pokus_s_cisly_df.groupby("klíč").aggregate(["min", numpy.median, max])

In [ ]:
pokus_s_cisly_df

Můžeš říct, že metoda `aggregate` se používá k aplikaci **jedné nebo více agregačních funkcí na seskupená data**.

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

| pobocka | ovoce | prodane_mnozstvi | cena |
| :- | :- | :- | :- |
| A | apple | 10 | 20 |

In [ ]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
ovoce_df = DataFrame(ovoce_data)

In [ ]:
ovoce_df

In [ ]:
ovoce_df.groupby('pobocka').aggregate({'prodane_mnozstvi': "sum"})  # .sum()

In [ ]:
ovoce_df.groupby('pobocka').aggregate({'prodane_mnozstvi': sum, 'cena': 'mean'})

1. Nejprve jsou hodnoty seskupení podle sloupce `pobocka`,
2. poté specifikuješ pomocí `aggregate` funkce a sloupce,
3. .. tedy sumarizovat hodnoty v `prodane_mnozstvi` a získat průměr `cena` pro každou pobočku.

### Metoda FILTER

---

Filtrování ti umožní zahodit takové údaje, které nesplňují zadanou podmínku.

Metoda `filter` se používá k vybrání **seskupených dat** podle splnění **určité podmínky**.

Představme si, že máme následující dataset s informacemi o prodeji ovoce v různých obchodech:

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [ ]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
ovoce_df = DataFrame(ovoce_data)

In [ ]:
ovoce_df

In [ ]:
def vyber_s_prodejem_nad_limit(data: DataFrame,
                               limit: int = 30) -> bool:
    return data['prodane_mnozstvi'].sum() > limit

<br>

Funkce `vyber_s_prodejem_nad_limit`, tedy **filtrovací funkce**, musí vracet **boolean** datový typ.

In [ ]:
ovoce_df.groupby('pobocka').filter(vyber_s_prodejem_nad_limit)

1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `filter`, která umožňuje použít uživatelem definovanou funkci (`vyber_pouze_pobocky_s_prodejem_nad_limit`),
3. definuješ funkci, která vybere pouze pobočky s větším prodejem než je parametr `limit`,
4. metoda `filter` pak vybere pouze ty řádky, které splňují tuto podmínku.

### Metoda TRANSFORM

---

Zatímco předchozí výsledky *agregace* pomocí `groupby` vraceli redukované množství dat.

Transformace obvykle vrací data o stejném rozsahu jako vstupní data. Jenom upravená.

Metoda `transform` se používá k aplikaci určité **transformační funkce** na každý prvek seskupených dat.

In [ ]:
ovoce_df

In [ ]:
def vrat_procenta_z_celkoveho_prodeje(udaj: Series) -> float:
    return round(udaj / udaj.sum() * 100, 1)

<br>

Vytvoření nového sloupečku `vrat_procenta_z_celkoveho_prodeje`:

In [ ]:
ovoce_df['Procento z celkoveho prodeje'] = ovoce_df.groupby('pobocka')['prodane_mnozstvi'] \
    .transform(vrat_procenta_z_celkoveho_prodeje)

In [ ]:
ovoce_df

1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `transform`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere vypočítá procentuální vyjádření prodaného zboží pro pobočku,
4. přidáš nový sloupeček `procento_z_celkoveho_prodeje`.

### Metoda `APPLY`

---

Metoda `apply` ti také dovolí, používat uživatelem definované funkce na seskupená data.

Následně vrací objekt knihovny `pandas` (buď `DataFrame`, nebo `Series`, a nebo skalární hodnota).

In [ ]:
ovoce_df

In [ ]:
def vypocitej_vydelek_za_artikl(seskupene: DataFrame):
    seskupene['vydelek_za_artikl'] = seskupene['prodane_mnozstvi'] * seskupene['cena']
    return seskupene

<br>

Metoda `apply` je **obecnější a flexibilnější** než `transform`.

`apply` umožňuje použít uživatelem definovanou funkci na každou skupinu po seskupení dat pomocí `groupby`.

Výsledek metody `apply` může mít jiný tvar než původní data.

In [ ]:
ovoce_df.groupby('pobocka', group_keys=False).apply(vypocitej_vydelek_za_artikl)

In [ ]:
# ovoce_df.groupby?

Od posledních verzí frameworku (`1.5.0` a vyšší) platí, že pokud bude výsledkem `DataFrame` nebo `Series` musíš uvést argument pro `group_keys=True`).

<br>

In [ ]:
vystup = ovoce_df.groupby('pobocka', group_keys=True).apply(vypocitej_vydelek_za_artikl)

In [ ]:
vystup

Na první pohled vypadají metody `apply` a `transform` docela podobně.

Metoda `transform` je trochu omezenější než `apply`.

Slouží k aplikaci uživatelem definované nebo vestavěné funkce **na každý prvek** skupiny po seskupení s `groupby`.

`transform` musí vracet hodnotu stejného tvaru jako vstupní data.

Výsledek metody `transform` **má stejný tvar jako původní data**.

#### Apply
* potřebuješ výsledek, který **má jiný tvar než původní data**,
* umí zpracovat **více sloupečků současně**.

#### Transform
* potřebuješ výsledek, který **má stejný tvar jako původní data**,
* umí zpracovat **pouze jeden sloupeček**.

In [ ]:
df_rozdily = DataFrame({
    'KLIC': ['A','B','C'] * 3,
    'A': numpy.arange(9),
    'B': [1,2,3] * 3,
})

In [ ]:
df_rozdily

#### `transform` vrací výsledky ve stejném tvaru

---

In [ ]:
def vypocitej_sumu(data):
    return data.sum()

In [ ]:
seskup_df_rozdily_apply = df_rozdily.groupby('KLIC')['A'].apply(vypocitej_sumu)

In [ ]:
seskup_df_rozdily_apply

In [ ]:
seskup_df_rozdily_trans = df_rozdily.groupby('KLIC')['A'].transform(vypocitej_sumu)

In [ ]:
seskup_df_rozdily_trans

#### `apply` umí pracovat s více sloupečky, `transform` jen s jedním

---

In [ ]:
df_rozdily

In [ ]:
def vypocitej_rozdil(data):
    return data['B'] - data['A']

In [ ]:
df_rozdily.groupby('KLIC').apply(vypocitej_rozdil)

In [ ]:
# df_rozdily.groupby('KLIC').transform(vypocitej_rozdil)

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí GroupBy a agregační funkce**

Máš zadaný takový datový set.

Následně:
1. Pomocí metody `filter` vyber prodejny, které prodaly **alespoň 30 produktů**,
2. na filtrovaném datasetu použijte metodu `apply` pro výpočet **celkového příjmu z prodeje pro každý obchod**.

In [ ]:
from pandas import DataFrame

df_prodej_hardware = DataFrame({
    'prodejna_id': (5, 4, 1, 5, 5, 1, 4, 2, 5, 1, 3, 1, 3, 4, 2, 1, 5, 4, 1, 5),
    'transakce_id': (1278, 1216, 1866, 1872, 1797, 1272, 1880, 1061, 1595, 1879, 1728,
       1341, 1396, 1698, 1018, 1176, 1611, 1395, 1444, 1232),
    'predmet_prodeje': ('grafická_karta', 'SSD', 'RAM', 'procesor', 'grafická_karta',
       'základní_deska', 'SSD', 'SSD', 'grafická_karta', 'RAM',
       'grafická_karta', 'procesor', 'grafická_karta', 'SSD',
       'grafická_karta', 'RAM', 'základní_deska', 'HDD', 'grafická_karta',
       'RAM'),
    'pocet_prodanych_ks': (1,  5,  6,  6,  3,  7,  9, 10,  8,  6,  8,  5,  8, 10,  4, 10,  8,
       10,  2,  5),
    'cena_predmetu': (19500.69874949, 19731.10951735, 14114.15342339, 10953.87914371,
        6535.78851758, 16369.00288429, 13852.2578648 ,  3671.03031723,
       18263.08009763, 16539.476237  , 19021.09830919, 14651.53041357,
       12461.59632075,  8655.73920767, 18688.2054254 , 17388.24584526,
        1381.76406707,  1014.1560027 ,  7841.03565412, 16305.78995025)
})

In [ ]:
def vyber_prodejny_s_vice_nez_triceti_produkty(
    data: DataFrame, limit: int = 30
):
    return data['pocet_prodanych_ks'].sum() >= limit

In [ ]:
hledane_prodejny = df_prodej_hardware.groupby('prodejna_id').filter(
    vyber_prodejny_s_vice_nez_triceti_produkty
)

In [ ]:
hledane_prodejny

In [ ]:
df_prodej_hardware.groupby('prodejna_id').sum()

In [ ]:
agregovano = df_prodej_hardware.groupby('prodejna_id')['pocet_prodanych_ks'].sum()
odfiltrovano = agregovano[agregovano > 30]

print(odfiltrovano)

In [ ]:
df_prodej_hardware.head()

<details>
    <summary>▶️ Řešení</summary>
    
```python
def vyber_prodej_vetsi_nez_limit(data, limit: int = 30):
    return data['pocet_prodanych_ks'].sum() > limit
    
def vypocitej_celkovy_vydelek_prodejny(skupina):
    return (skupina['pocet_prodanych_ks'] * skupina['cena_predmetu']).sum()

df_filtr_hardware = df_prodej_hardware.groupby('prodejna_id') \
                        .filter(vyber_prodej_vetsi_nez_limit)

#         cilove_prodejny = df_prodej_hardware.groupby('prodejna_id') \
# .filter(lambda prodejna: prodejna.pocet_prodanych_ks.sum() >= 30)


celkovy_vydelek = df_filtr_hardware.groupby('prodejna_id') \
                        .apply(vypocitej_celkovy_vydelek_prodejny)

# prodeje_prodejen = cilove_prodejny.groupby('prodejna_id') \
# .aggregate({'pocet_prodanych_ks': sum})

# df_prodej_hardware["prodejna_id"].unique()

# prijmy_prodejen = pd.DataFrame(df_prodej_hardware.groupby('prodejna_id')['cena_predmetu'] \
#                 .apply(lambda prodejna: prodejna.sum())) \
#                 .rename(columns = { 'cena_predmetu': 'prijem_z_prodeje' } ) \
#                 .round( 2 )
```
</details>


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.P16hnt14CM-mJ5eAKykxLAHaHa%26pid%3DApi&f=1&ipt=27e3f5b23fe44567ad0730192d5dcb13c7c23aa6e87579d16df05e27461dc12c&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

## Pivot tabulky

---


*Pivot tabulky* (*kontingenční tabulky*) jsou užitečné pro přehledné zobrazení a analýzu dat z tabulkových zdrojů.

Pomocí pivot tabulek můžeš **seskupit data podle určitých kategorií** a provést agregaci hodnot.

Nejprve si představ situaci bez pivot tabulek, pomocí ukázky níže:

In [ ]:
!pip install seaborn

In [ ]:
import seaborn

<br>

V této ukazce použiješ vzorová data týkající se [nehody lodi Titanic](https://en.wikipedia.org/wiki/Sinking_of_the_Titanic):

In [ ]:
df_titanic = seaborn.load_dataset('titanic')

In [ ]:
df_titanic.info()

<br>

Pro jakoukoliv pokročilou analýzu dat, potřebuješ údaj seskupit.

Tak jak jsi do prováděl doposud, pomocí `groupby`:

In [ ]:
df_titanic.head()

In [ ]:
df_titanic["sex"].unique()

In [ ]:
df_titanic.groupby('sex')

<br>

Potřebuješ průměrnou hodnotu, ať je na sloupečku `survived` vidět poměr přeživších:

In [ ]:
df_titanic.groupby('sex')['survived'].mean()

<br>

Pro lepší čitelnost **zaokrouhlím výsledek na dvě desetinné čísla**:

In [ ]:
df_titanic.groupby('sex')['survived'].mean().round(2)

Takový průzkum z této studie ti dá jasný pohled na věc:
* 3 ze 4 žen přežily,
* 1 z 5 mužů přežil.

<br>

Pokud budeš potřebovat detailnější analýzy, budeš potřebovat více dat.

Třeba situaci, kde kromě pohlaví, bereš v potaz **třídu cestujících** `class`:

In [ ]:
df_titanic["class"].unique()

In [ ]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean()

In [ ]:
type(df_titanic.groupby(['sex', 'class'])[['survived']])

In [ ]:
type(df_titanic.groupby(['sex', 'class'])['survived'].mean())

In [ ]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean().round(2)

<br>

Metodou `unstack` si můžeš vytvořit nové sloupečky, které jsou postavené na novém Indexu, nebo Indexech (*Multiindex*):

In [ ]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

Takový průzkum ti dá skutečně lepší pohled na věc.

Současně ale roste **náročnost ohlášení**.

Zápis "bobtná" a **stává se náročnějším na přečtení a pochopení**.

<br>

### Pivot tabulka

---

Podobné řešení ti nabízí funkce `pivot_table`:

In [ ]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

In [ ]:
df_titanic.pivot_table('survived', index='sex', columns='class').round(2)

In [ ]:
df_titanic.index

Zásádním rozdílem je ovšem **čitelnost**, kterou máš pro tuto variantu zápisu.

```
Clarity beats purity
```

Pomocí vhodných argumentů, můžeš doplnit vysvětlivky tam, kde funkce `groupby` nemohla.

Stejně platí, že pokud budeš potřebovat **další Index**, můžeš si pomoci funkcí `cut`:

In [ ]:
df_titanic.head()

In [ ]:
df_titanic['age'].describe()

In [ ]:
from pandas import cut

In [ ]:
age = cut(df_titanic['age'], [1, 40, 80])

In [ ]:
age  # < xxx , xxx )

In [ ]:
vystup_df = df_titanic.pivot_table('survived', index=['sex', age], columns='class').round(2)

In [ ]:
vystup_df

In [ ]:
vystup_df.index

Pomocí funkce `cut` doplníš tabulku na **MultiIndex**.

Ten nyní vytvoří i rozsah pro věkovou kategorii.

<br>

**🧠 CVIČENÍ 🧠, procvič si pivot tabulky**

Ze zadaného datasetu vytvoř **pivot tabulku**, která zobrazí **počet prodaných kusů ovoce** pro **každý obchod** a **druh ovoce**.

In [ ]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
# pivot_table()
#              prodane_mnozstvi      jablko banan
# Pobocka
#    A 
#    B
#    C 

In [ ]:
ovoce_df = DataFrame(ovoce_data)

In [ ]:
ovoce_df

In [ ]:
pt = ovoce_df.pivot_table(values='prodane_mnozstvi',
                     index='pobocka',
                     columns='ovoce')

In [ ]:
pt.index.names = ['pobocka_id']

In [ ]:
pt

<details>
    <summary>▶️ Řešení</summary>
    
```python
pivot_tabulka = df_ovoce.pivot_table(values="prodane_mnozstvi", index="pobocka", columns="ovoce")
```
</details>

---